In [0]:
from google.colab import drive
drive.mount('/content/gdrive')

Go to this URL in a browser: https://accounts.google.com/o/oauth2/auth?client_id=947318989803-6bn6qk8qdgf4n4g3pfee6491hc0brc4i.apps.googleusercontent.com&redirect_uri=urn%3Aietf%3Awg%3Aoauth%3A2.0%3Aoob&scope=email%20https%3A%2F%2Fwww.googleapis.com%2Fauth%2Fdocs.test%20https%3A%2F%2Fwww.googleapis.com%2Fauth%2Fdrive%20https%3A%2F%2Fwww.googleapis.com%2Fauth%2Fdrive.photos.readonly%20https%3A%2F%2Fwww.googleapis.com%2Fauth%2Fpeopleapi.readonly&response_type=code

Enter your authorization code:
··········
Mounted at /content/gdrive


In [0]:
# http://pytorch.org/
from os.path import exists
from wheel.pep425tags import get_abbr_impl, get_impl_ver, get_abi_tag
platform = '{}{}-{}'.format(get_abbr_impl(), get_impl_ver(), get_abi_tag())
cuda_output = !ldconfig -p|grep cudart.so|sed -e 's/.*\.\([0-9]*\)\.\([0-9]*\)$/cu\1\2/'
accelerator = cuda_output[0] if exists('/dev/nvidia0') else 'cpu'

!pip install -q http://download.pytorch.org/whl/{accelerator}/torch-0.4.1-{platform}-linux_x86_64.whl torchvision
import torch

tcmalloc: large alloc 1073750016 bytes == 0x57b4a000 @  0x7f0d90dc02a4 0x591a07 0x5b5d56 0x502e9a 0x506859 0x502209 0x502f3d 0x506859 0x504c28 0x502540 0x502f3d 0x506859 0x504c28 0x502540 0x502f3d 0x506859 0x504c28 0x502540 0x502f3d 0x507641 0x502209 0x502f3d 0x506859 0x504c28 0x502540 0x502f3d 0x507641 0x504c28 0x502540 0x502f3d 0x507641


In [0]:
import warnings
warnings.filterwarnings("ignore")

***

# START:

In [0]:
import numpy as np
import pandas as pd

from collections import Counter

import torch
import torch.nn as nn

from torch.autograd import Variable
from torch.utils.data import Dataset, DataLoader

### DEVICE:

In [0]:
device = 'cuda' if torch.cuda.is_available() else 'cpu'
device

'cuda'

### PATH:

In [0]:
path='gdrive/My Drive/netflix_colab/data/'

***

## FUNCTONS:

### 1. FN FOR TRAINING & VALIDATION:

In [0]:
def fn_train_eval(model, train_dataset, test_dataset, batch_size, optimizer, criterion, n_epochs):
    
  
    # INNER FUNCTIONS-----------------------------------------------
    
    def fn_train(model, iterator, optimizer, criterion):

        epoch_loss = 0
        model.train()
        for batch in iterator:
            
            movies  = Variable(batch[0][:, 0]).to(device)
            users   = Variable(batch[0][:, 1]).to(device)
            ratings = Variable(batch[1]).to(device)
            
            optimizer.zero_grad()                   # INITIALIZE
            
            predictions = model(users, movies)      # PREDICT
            loss = criterion(predictions, ratings)  # COMPUTE LOSS

            loss.backward()                         # BACK PROP
            optimizer.step()                        # GRADIENT DESCENT

            epoch_loss += loss.item()

        epoch_loss = epoch_loss/len(iterator)

        return epoch_loss 


    def fn_evaluate(model, iterator, criterion):

        epoch_loss = 0
        model.eval()
        with torch.no_grad():

            for batch in iterator:
            
                movies  = Variable(batch[0][:, 0]).to(device)
                users   = Variable(batch[0][:, 1]).to(device)
                ratings = Variable(batch[1]).to(device)


                predictions = model(users, movies)
                loss = criterion(predictions, ratings)

                epoch_loss += loss.item()

        epoch_loss = epoch_loss/len(iterator)

        return epoch_loss 
    
    
    # RUN--------------------------------------------------------------------

    train_iterator = DataLoader(dataset=train_dataset, batch_size=batch_size, shuffle=True, num_workers=2)
    test_iterator  = DataLoader(dataset=test_dataset,  batch_size=batch_size, shuffle=True, num_workers=2)

    listO_train_losses, listO_test_losses = [], []
    
    c1, c2 = 'print', 'print'
    for epoch in range(n_epochs):

        train_loss = fn_train(model, train_iterator, optimizer, criterion)
        if c1 == 'print':
            print('TRAINING >>')
            c1 = 'no_print'

        test_loss = fn_evaluate(model, test_iterator, criterion)
        if c2 == 'print':
            print('EVALUATING >>')
            c2 = 'no_print'

        listO_train_losses.append(train_loss)
        listO_test_losses.append(test_loss)
        
        print_params = [epoch+1, train_loss, test_loss]
        print('Epoch: {:03}  |  Train Loss: {:.3f}  | test Loss: {:.3f}  |'.format(*print_params))
        
    print()
    print('RETURNED: listO_train_losses, listO_test_losses, model')
    
    return listO_train_losses, listO_test_losses, model 

### 2. PLOTTING FN:

In [0]:
def retrn_model_plots(listO_train_losses, listO_test_losses):
    
    import pylab
    pylab.rcParams["figure.figsize"] = (10, 5)

    xs = list(range(1, len(listO_train_losses)+1))
    ys = listO_train_losses
    ys_v = listO_test_losses


    pylab.plot(xs, ys, '-', label = 'train')
    pylab.plot(xs, ys_v, '-', label = 'test')

    pylab.legend()
    pylab.xlabel('EPOCHS')
    pylab.ylabel('LOSS')
    
    pylab.show()

***

### CHECK DATA:

In [0]:
df_train = pd.read_csv(path + 'df_final_trainset.csv', index_col = 0)
df_train.head()

,movie,user,rating
333064,0,0,3
911833,1,1,5
1287538,2,2,3
941269,3,3,3
1694504,4,4,4


In [0]:
df_train.movie.max(), df_train.user.max()

(4997, 49799)

***

# CREATE - TRAIN & TEST PYTORCH DATASET CLASSES:

In [0]:
class RecommenderData(Dataset):

    def __init__(self, path, csv_file_name):
        
        df = pd.read_csv(path + csv_file_name, index_col = 0)
                        
        self.len = df.shape[0]
        
        self.x_data = torch.from_numpy(df.iloc[:, :-1].values).type(torch.LongTensor)
        self.y_data = torch.from_numpy(df.iloc[:, -1].values).type(torch.FloatTensor)

    def __getitem__(self, index):
        return self.x_data[index], self.y_data[index]

    def __len__(self):
        return self.len

In [0]:
path = path

train_dataset = RecommenderData(path, 'df_final_trainset.csv')
test_dataset  = RecommenderData(path, 'df_final_testset.csv')

In [0]:
train_dataset.x_data

tensor([[    0,     0],
        [    1,     1],
        [    2,     2],
        ...,
        [ 3652,  8903],
        [   56, 10623],
        [  263, 39396]])

In [0]:
train_dataset.y_data

tensor([3., 5., 3.,  ..., 1., 4., 5.])

In [0]:
train_dataset.x_data[:, 0].max(), train_dataset.x_data[:, 1].max()

(tensor(4997), tensor(49799))

In [0]:
train_dataset.len, test_dataset.len

(2350408, 587329)

***

# MODEL:

In [0]:
class MatrixFactorization_2(torch.nn.Module):
    
    def __init__(self, n_users, n_items, embedding_dim, hidden_dim, last_dim):
        super().__init__()
        
	    # create user & item embeddings of same size:
        self.user_embeddings = nn.Embedding(n_users, embedding_dim, sparse=False)
        self.item_embeddings = nn.Embedding(n_items, embedding_dim, sparse=False)
        
        self.weights1 = nn.Linear(embedding_dim, hidden_dim)
        self.weights2 = nn.Linear(hidden_dim, last_dim)
        
        self.batch_norm1 = nn.BatchNorm1d(embedding_dim)
        self.batch_norm2 = nn.BatchNorm1d(hidden_dim)
        
        self.dropout1 = nn.Dropout2d(p=0.5)
        self.dropout2 = nn.Dropout2d(p=0.25)
        
        self.relu = nn.ReLU()

    def forward(self, user, item):
        
    	# CREATING USER LATENT VARIABLES:
        # layer 1
        user_vec = self.user_embeddings(user)
        user_vec = self.batch_norm1(user_vec) 
        user_vec = self.dropout1(user_vec) 
        user_vec = self.weights1(user_vec) 
        user_vec = self.relu(user_vec)
        # layers 2 
        user_vec = self.batch_norm2(user_vec) 
        user_vec = self.dropout2(user_vec) 
        user_vec = self.weights2(user_vec) 
        user_vec = self.relu(user_vec)
        
        
        # CREATING ITEM LATENT VARIABLES:
        # layer 1
        item_vec = self.item_embeddings(item)
        item_vec = self.batch_norm1(item_vec) 
        item_vec = self.dropout1(item_vec) 
        item_vec = self.weights1(item_vec) 
        item_vec = self.relu(item_vec)
        # layers 2 
        item_vec = self.batch_norm2(item_vec) 
        item_vec = self.dropout2(item_vec) 
        item_vec = self.weights2(item_vec) 
        item_vec = self.relu(item_vec)
        
        y_pred = (user_vec * item_vec).sum(1)
        
        return y_pred   

***

# MODEL TRAINING & EVALUATION:

### TRIAL 1: Embedding_dim, hidden_dim, last_dim = 100, 50, 5

In [0]:
 n_users, n_items, embedding_dim, hidden_dim, last_dim = 50000, 5000, 100, 50, 5

model = MatrixFactorization_2(n_users, n_items, embedding_dim, hidden_dim, last_dim)

learning_rate = 1e-3
optimizer = torch.optim.Adam(model.parameters(), lr=learning_rate)
criterion = nn.MSELoss()

model = model.to(device)
criterion = criterion.to(device)

train_dataset = train_dataset
test_dataset = test_dataset
batch_size = 1000
n_epochs = 50

%time listO_train_losses, listO_test_losses, model = fn_train_eval(model, train_dataset, test_dataset, batch_size, optimizer, criterion, n_epochs)

TRAINING >>
EVALUATING >>
Epoch: 001  |  Train Loss: 1.665  | test Loss: 1.091  |
Epoch: 002  |  Train Loss: 1.074  | test Loss: 0.972  |
Epoch: 003  |  Train Loss: 0.985  | test Loss: 0.925  |
Epoch: 004  |  Train Loss: 0.942  | test Loss: 0.903  |
Epoch: 005  |  Train Loss: 0.917  | test Loss: 0.891  |
Epoch: 006  |  Train Loss: 0.901  | test Loss: 0.882  |
Epoch: 007  |  Train Loss: 0.891  | test Loss: 0.879  |
Epoch: 008  |  Train Loss: 0.883  | test Loss: 0.875  |
Epoch: 009  |  Train Loss: 0.877  | test Loss: 0.870  |
Epoch: 010  |  Train Loss: 0.872  | test Loss: 0.870  |
Epoch: 011  |  Train Loss: 0.868  | test Loss: 0.868  |
Epoch: 012  |  Train Loss: 0.865  | test Loss: 0.871  |
Epoch: 013  |  Train Loss: 0.863  | test Loss: 0.867  |
Epoch: 014  |  Train Loss: 0.861  | test Loss: 0.867  |
Epoch: 015  |  Train Loss: 0.859  | test Loss: 0.865  |
Epoch: 016  |  Train Loss: 0.857  | test Loss: 0.866  |
Epoch: 017  |  Train Loss: 0.856  | test Loss: 0.866  |
Epoch: 018  |  Train L

***

### TRIAL 2:  Embedding_dim, hidden_dim, last_dim = 1000, 500, 100

In [0]:
 n_users, n_items, embedding_dim, hidden_dim, last_dim = 50000, 5000, 1000, 500, 100

model = MatrixFactorization_2( n_users, n_items, embedding_dim, hidden_dim, last_dim)

learning_rate = 5e-3
optimizer = torch.optim.Adam(model.parameters(), lr=learning_rate)
criterion = nn.MSELoss()

model = model.to(device)
criterion = criterion.to(device)

train_dataset = train_dataset
test_dataset = test_dataset
batch_size = 1000
n_epochs = 50

%time listO_train_losses, listO_test_losses, model = fn_train_eval(model, train_dataset, test_dataset, batch_size, optimizer, criterion, n_epochs)

TRAINING >>
EVALUATING >>
Epoch: 001  |  Train Loss: 1.046  | test Loss: 0.882  |
Epoch: 002  |  Train Loss: 0.888  | test Loss: 0.876  |
Epoch: 003  |  Train Loss: 0.867  | test Loss: 0.868  |
Epoch: 004  |  Train Loss: 0.860  | test Loss: 0.866  |
Epoch: 005  |  Train Loss: 0.855  | test Loss: 0.868  |
Epoch: 006  |  Train Loss: 0.853  | test Loss: 0.871  |
Epoch: 007  |  Train Loss: 0.840  | test Loss: 0.837  |
Epoch: 008  |  Train Loss: 0.808  | test Loss: 0.825  |
Epoch: 009  |  Train Loss: 0.793  | test Loss: 0.825  |
Epoch: 010  |  Train Loss: 0.786  | test Loss: 0.816  |
Epoch: 011  |  Train Loss: 0.782  | test Loss: 0.821  |
Epoch: 012  |  Train Loss: 0.779  | test Loss: 0.815  |
Epoch: 013  |  Train Loss: 0.777  | test Loss: 0.816  |
Epoch: 014  |  Train Loss: 0.775  | test Loss: 0.814  |
Epoch: 015  |  Train Loss: 0.774  | test Loss: 0.811  |
Epoch: 016  |  Train Loss: 0.772  | test Loss: 0.814  |
Epoch: 017  |  Train Loss: 0.771  | test Loss: 0.812  |
Epoch: 018  |  Train L

***

### TRIAL 3: Embedding_dim, hidden_dim, last_dim = 500, 100, 10

In [0]:
 n_users, n_items, embedding_dim, hidden_dim, last_dim = 50000, 5000, 500, 100, 10

model = MatrixFactorization_2( n_users, n_items, embedding_dim, hidden_dim, last_dim)

learning_rate = 5e-3
optimizer = torch.optim.Adam(model.parameters(), lr=learning_rate)
criterion = nn.MSELoss()

model = model.to(device)
criterion = criterion.to(device)

train_dataset = train_dataset
test_dataset = test_dataset
batch_size = 1000
n_epochs = 50

%time listO_train_losses, listO_test_losses, model = fn_train_eval(model, train_dataset, test_dataset, batch_size, optimizer, criterion, n_epochs)

TRAINING >>
EVALUATING >>
Epoch: 001  |  Train Loss: 1.064  | test Loss: 0.887  |
Epoch: 002  |  Train Loss: 0.882  | test Loss: 0.869  |
Epoch: 003  |  Train Loss: 0.869  | test Loss: 0.870  |
Epoch: 004  |  Train Loss: 0.861  | test Loss: 0.871  |
Epoch: 005  |  Train Loss: 0.856  | test Loss: 0.865  |
Epoch: 006  |  Train Loss: 0.851  | test Loss: 0.860  |
Epoch: 007  |  Train Loss: 0.834  | test Loss: 0.836  |
Epoch: 008  |  Train Loss: 0.810  | test Loss: 0.821  |
Epoch: 009  |  Train Loss: 0.796  | test Loss: 0.818  |
Epoch: 010  |  Train Loss: 0.787  | test Loss: 0.820  |
Epoch: 011  |  Train Loss: 0.779  | test Loss: 0.824  |
Epoch: 012  |  Train Loss: 0.772  | test Loss: 0.807  |
Epoch: 013  |  Train Loss: 0.764  | test Loss: 0.807  |
Epoch: 014  |  Train Loss: 0.758  | test Loss: 0.800  |
Epoch: 015  |  Train Loss: 0.754  | test Loss: 0.804  |
Epoch: 016  |  Train Loss: 0.750  | test Loss: 0.799  |
Epoch: 017  |  Train Loss: 0.748  | test Loss: 0.800  |
Epoch: 018  |  Train L

***

### TRIAL 4:  Embedding_dim, hidden_dim, last_dim = 500, 250, 5

In [0]:
 n_users, n_items, embedding_dim, hidden_dim, last_dim = 50000, 5000, 500, 250, 5

model = MatrixFactorization_2( n_users, n_items, embedding_dim, hidden_dim, last_dim)

learning_rate = 5e-3
optimizer = torch.optim.Adam(model.parameters(), lr=learning_rate)
criterion = nn.MSELoss()

model = model.to(device)
criterion = criterion.to(device)

train_dataset = train_dataset
test_dataset = test_dataset
batch_size = 1000
n_epochs = 50

%time listO_train_losses, listO_test_losses, model = fn_train_eval(model, train_dataset, test_dataset, batch_size, optimizer, criterion, n_epochs)

TRAINING >>
EVALUATING >>
Epoch: 001  |  Train Loss: 1.065  | test Loss: 0.886  |
Epoch: 002  |  Train Loss: 0.882  | test Loss: 0.870  |
Epoch: 003  |  Train Loss: 0.868  | test Loss: 0.871  |
Epoch: 004  |  Train Loss: 0.861  | test Loss: 0.867  |
Epoch: 005  |  Train Loss: 0.857  | test Loss: 0.866  |
Epoch: 006  |  Train Loss: 0.846  | test Loss: 0.843  |
Epoch: 007  |  Train Loss: 0.816  | test Loss: 0.824  |
Epoch: 008  |  Train Loss: 0.799  | test Loss: 0.827  |
Epoch: 009  |  Train Loss: 0.789  | test Loss: 0.813  |
Epoch: 010  |  Train Loss: 0.782  | test Loss: 0.817  |
Epoch: 011  |  Train Loss: 0.773  | test Loss: 0.810  |
Epoch: 012  |  Train Loss: 0.765  | test Loss: 0.802  |
Epoch: 013  |  Train Loss: 0.759  | test Loss: 0.805  |
Epoch: 014  |  Train Loss: 0.755  | test Loss: 0.799  |
Epoch: 015  |  Train Loss: 0.751  | test Loss: 0.802  |
Epoch: 016  |  Train Loss: 0.748  | test Loss: 0.799  |
Epoch: 017  |  Train Loss: 0.745  | test Loss: 0.803  |
Epoch: 018  |  Train L

***

### TRIAL 5:  Embedding_dim, hidden_dim, last_dim = 500, 250, 15

In [0]:
 n_users, n_items, embedding_dim, hidden_dim, last_dim = 50000, 5000, 500, 250, 15

model = MatrixFactorization_2( n_users, n_items, embedding_dim, hidden_dim, last_dim)

learning_rate = 5e-3
optimizer = torch.optim.Adam(model.parameters(), lr=learning_rate)
criterion = nn.MSELoss()

model = model.to(device)
criterion = criterion.to(device)

train_dataset = train_dataset
test_dataset = test_dataset
batch_size = 1000
n_epochs = 50

%time listO_train_losses, listO_test_losses, model = fn_train_eval(model, train_dataset, test_dataset, batch_size, optimizer, criterion, n_epochs)

TRAINING >>
EVALUATING >>
Epoch: 001  |  Train Loss: 1.056  | test Loss: 0.880  |
Epoch: 002  |  Train Loss: 0.884  | test Loss: 0.876  |
Epoch: 003  |  Train Loss: 0.869  | test Loss: 0.875  |
Epoch: 004  |  Train Loss: 0.862  | test Loss: 0.867  |
Epoch: 005  |  Train Loss: 0.857  | test Loss: 0.867  |
Epoch: 006  |  Train Loss: 0.837  | test Loss: 0.841  |
Epoch: 007  |  Train Loss: 0.809  | test Loss: 0.819  |
Epoch: 008  |  Train Loss: 0.796  | test Loss: 0.820  |
Epoch: 009  |  Train Loss: 0.789  | test Loss: 0.815  |
Epoch: 010  |  Train Loss: 0.783  | test Loss: 0.813  |
Epoch: 011  |  Train Loss: 0.774  | test Loss: 0.806  |
Epoch: 012  |  Train Loss: 0.767  | test Loss: 0.805  |
Epoch: 013  |  Train Loss: 0.760  | test Loss: 0.810  |
Epoch: 014  |  Train Loss: 0.755  | test Loss: 0.799  |
Epoch: 015  |  Train Loss: 0.752  | test Loss: 0.800  |
Epoch: 016  |  Train Loss: 0.749  | test Loss: 0.801  |
Epoch: 017  |  Train Loss: 0.746  | test Loss: 0.802  |
Epoch: 018  |  Train L

## BEST PERFORMANCE : TRIAL 3 - Epoch: 029  |  Train Loss: 0.722  | test Loss: 0.797